# Convolutional Neural Network

## Setup and Context

### Introduction

The following is a Convolutional Neural Network that differentiates between cats and dogs. The dataset used is from [Kaggle](https://www.kaggle.com/datasets/tongpython/cat-and-dog) and contains over 10k images.

### Import Statements

In [1]:
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
tf.__version__

'2.15.0'

## Data Preprocessing

### The Training Set

We want to perform some **transformations** (such as rotation, flip, zoom, translation, shear e.t.c) on the images of the training set. The idea behind these transformations is to expose the model to a diverse range of input variations during training, which helps prevent it from memorizing specific details of the training set. This way we can avoid **overfitting**.

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

The `rescale` parameter of the `ImageDataGenerator` will apply **feature scaling** to every pixel. By scaling the pixel values by 1/255, we normalize them to the range [0, 1].

Below is a generator that will read images found in subfolers of 'data/train', and indefinitely generate batches of augmented image data.

In [4]:
training_set = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='binary') # this is a binary classification problem i.e either cats or dogs

Found 8005 images belonging to 2 classes.


### The Validation Set

We do not want to perform transformations on the validation data to avoid **data leakage**. We want to ensure that the evaluation of the model's performance reflects its ability to generalize to new, unseen data in a real-world scenario. We do however still need to normalize the data for our neural network.

In [5]:
val_datagen = ImageDataGenerator(rescale=1./255)

Similary to the training, we create a generator for the validation set. Only difference is the directory of the images.

In [6]:
val_set = val_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 2023 images belonging to 2 classes.


## Building the Architecture

### Initializing the CNN

The Convolutional Neural Network is a sequence of layers. Therefore we are going to intialize our model with the `Sequential` class.

In [7]:
cnn = tf.keras.models.Sequential()

### Convolution Layer

We add the Convolution Layer with the `Conv2D` class to our model while specifying `filters` (number of feature detectors you want to apply to your images), `kernel_size` (the size of the feature detectors), `activation` (the activation function) and `input_shape` (the input shape).

When we add the very first layer, whether a convolution layer or a dense layer, we have to specify the input shape of our inputs. The input shape is (150, 150, 3) as the size of our images after preprocessing is 150x150 and we are using coloured images, 3 dimensions corresponding to the RGB channels.

For other layers after the input layer, we do not specify the input shape.

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(150, 150, 3)))

### Pooling Layer

Next we add a layer for pooling. More specifically, **max pooling**. The `MaxPool2D` class has two necessary parameters: `pool_size` (size of the pooling window) and `strides` (step size of the pooling window).

In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a Second Convolution Layer

Using a second convolutional layer in a Convolutional Neural Network allows the model to capture higher-level features by learning more complex patterns and representations from the input data.

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Flattening

We follow up by flattening the results of our convolutions and poolings into a one-dimensional vector.

In [11]:
cnn.add(tf.keras.layers.Flatten())

Now we can pass it on to a fully connected Neural Network.

### Full Connection

Join up to fully connected layers. Because of the complexity of computer vision, the hidden layer(s) can have large numbers of neurons.

In [12]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Output Layer

Because we are doing a binary classification problem, we only need 1 neuron in the output layer.

In [13]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

It is not recommended to use ReLU as the activation function of the output layer. Instead, the Sigmoid function would work better in this binary classification problem. If we where doing a multi-class classification problem, then Softmax would be the choice.

## Training

### Compiling the Neural Network

By compiling, we mean connecting the Neural Network to an optimizer, loss function and some metrics. We are using an **Atom Optimizer** to perform **Stochastic Gradient Descent**. Our **Loss Function** is the **Binary Cross Entropy Loss**. We also using **Accuracy Metrics** as this is the most relevant way to measure a classification model.

In [14]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training and Evaluating

Using our training set, we train the neural network and use the test set to evaluate its performance. For this example we will run 30 **epochs**.

In [15]:
cnn.fit(x=training_set, validation_data=val_set, epochs=30)

Epoch 1/30
251/251 [==============================] - 55s 216ms/step - loss: 0.7059 - accuracy: 0.5635 - val_loss: 0.6703 - val_accuracy: 0.5769
Epoch 2/30
251/251 [==============================] - 52s 207ms/step - loss: 0.6598 - accuracy: 0.6125 - val_loss: 0.6156 - val_accuracy: 0.6683
Epoch 3/30
251/251 [==============================] - 53s 211ms/step - loss: 0.6335 - accuracy: 0.6356 - val_loss: 0.6158 - val_accuracy: 0.6619
Epoch 4/30
251/251 [==============================] - 52s 208ms/step - loss: 0.6223 - accuracy: 0.6548 - val_loss: 0.5867 - val_accuracy: 0.6896
Epoch 5/30
251/251 [==============================] - 52s 208ms/step - loss: 0.6026 - accuracy: 0.6785 - val_loss: 0.5528 - val_accuracy: 0.7182
Epoch 6/30
251/251 [==============================] - 52s 206ms/step - loss: 0.5933 - accuracy: 0.6814 - val_loss: 0.5799 - val_accuracy: 0.6851
Epoch 7/30
251/251 [==============================] - 51s 202ms/step - loss: 0.5658 - accuracy: 0.7026 - val_loss: 0.5323 - val_ac

## Making Predictions

Time to make a prediction. We have an example image that we are going to check.

<p align='center'>
<img src='./assets/images/cnn-dog-test-1.jpg'>
</p>

<p align='center'>An image of a dog</p>

Firstly, we load the image. The image MUST be the same size as the ones used during training.

In [16]:
test_image = image.load_img('./assets/images/cnn-dog-test-1.jpg', target_size=(150, 150))

Our test image must also be converted into a 2D-array as this is the input that is expected by our Neural Network.

In [17]:
test_image = image.img_to_array(test_image)

In CNNs, the input usually expects a batch of images, even if you're predicting on just one image. The extra dimension represents the batch size. The shape of the input expected by the CNN should be in the format (batch_size, height, width, channels) So we must add the batch_size as the first dimension.

In [18]:
test_image = np.expand_dims(test_image, axis=0)

Now we normalize the the test image and predict.

In [19]:
result = cnn.predict(test_image / 255.0)

1/1 [==============================] - 0s 202ms/step


Our model would give us a value in the range [0, 1]. Let us see what each end represents.

In [20]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

So a value of 0 would mean that the model is 100% certain it is a cat and a value of 1 would be that the model is 100% certain it is a dog.

Let us see the result of our prediction. Recall that our result would be a single batch of only one element.

In [21]:
if result[0][0] > 0.5:
    prediction = "dog"
else:
    prediction = "cat"

prediction

'dog'

### More Examples

<p align='center'>
<img src='./assets/images/cnn-cat-test-1.jpg'>
</p>

In [22]:
test_image = image.load_img('./assets/images/cnn-cat-test-1.jpg', target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = cnn.predict(test_image / 255.0)
if result[0][0] > 0.5:
    prediction = "dog"
else:
    prediction = "cat"

prediction

1/1 [==============================] - 0s 24ms/step


'cat'

<p align='center'>
<img src='./assets/images/cnn-dog-test-2.jpg'>
</p>

In [23]:
test_image = image.load_img('./assets/images/cnn-dog-test-2.jpg', target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = cnn.predict(test_image / 255.0)
if result[0][0] > 0.5:
    prediction = "dog"
else:
    prediction = "cat"

prediction

1/1 [==============================] - 0s 24ms/step


'dog'

<p align='center'>
<img src='./assets/images/cnn-cat-test-2.jpg'>
</p>

In [24]:
test_image = image.load_img('./assets/images/cnn-cat-test-2.jpg', target_size=(150, 150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = cnn.predict(test_image / 255.0)
if result[0][0] > 0.5:
    prediction = "dog"
else:
    prediction = "cat"

prediction

1/1 [==============================] - 0s 41ms/step


'cat'